In [ ]:
# ipython-sql from: https://github.com/catherinedevlin/ipython-sql 
# downloaded 03/10/2017
# setup.py to install (note: latest release w/ pip install was missing features)

In [ ]:
import pysandag.database as sdb
import pandas as pd

In [ ]:
%load_ext sql

In [ ]:
# connect to database
[postgres,user,password,db] = sdb.get_connection_string("../configs/dbconfig.yml", 'urbansim_database').split(":")
%sql $postgres:$user:$password:$db

In [ ]:
# get all jurisdictions for outer join - since some jurisdictions do not have sch developments
jurisdictions = %sql SELECT jurisdiction_id,name as jurisdiction FROM ref.jurisdiction j WHERE jurisdiction_id < 20
jurisdictions_df = jurisdictions.DataFrame()

In [ ]:
# urbansim output 
# get all scheduled developemnt events built during the current scenario
# output where new buildings are scheduled development events (sch_dev = 1)
# using phase-in spreadsheet
scenario_id = 250 # from urbansim output

In [ ]:
%%sql 
sch_dev <<
SELECT b.jurisdiction_id, new_units,building_type_id 
FROM urbansim_output.buildings b WHERE scenario_id = :scenario_id AND 
sch_dev = 1 ORDER BY b.jurisdiction_id

In [ ]:
sch_dev_df = sch_dev.DataFrame()

In [ ]:
# join with jurisdictions_df to get jurisdiction names 
sched_dev_by_jurisdiction = pd.merge(sch_dev_df, jurisdictions_df, on='jurisdiction_id', how='outer')

In [ ]:
# fill NAs for jurisdictions with no sched developments
sched_dev_by_jurisdiction['new_units'].fillna(0, inplace=True) 
sched_dev_by_jurisdiction['building_type_id'].fillna(19, inplace=True)

In [ ]:
# rename building type id
sched_dev_by_jurisdiction['building_type_id'].replace(19, 'single-family',inplace=True)
sched_dev_by_jurisdiction['building_type_id'].replace(21, 'multi-family',inplace=True)

In [ ]:
# sum num of units by jurisdiction and building type 
schdev_grp1 =sched_dev_by_jurisdiction.groupby(['jurisdiction','building_type_id'])[['new_units']].sum()
schdev_grp1.reset_index(drop=False,inplace=True)

In [ ]:
# pivot table so that single-family and multi-family are column headers
sched_dev_pivot = schdev_grp1.pivot_table(index = ['jurisdiction'],values=['new_units'], columns='building_type_id')

In [ ]:
# remove column names (building type id), drop multiindex (new units), reset index
sched_dev_pivot.columns.names = (None,None)
sched_dev_pivot.columns = sched_dev_pivot.columns.droplevel()
sched_dev_pivot.reset_index(inplace=True)

In [ ]:
# fill NAs (where no single-family or no multi-family for a jurisdiction)
sched_dev_pivot.fillna(0, inplace=True)

In [ ]:
# calculate total units
sched_dev_pivot['total_units'] = sched_dev_pivot['multi-family'] + sched_dev_pivot['single-family']

In [ ]:
# reorder columns for printing
sched_dev_pivot = sched_dev_pivot[['jurisdiction','single-family','multi-family','total_units']]

In [ ]:
# calculate totals for region and append to dataframe
sched_dev_pivot_for_region = pd.DataFrame([['Region',\
                                            sched_dev_pivot['single-family'].sum(),\
                                            sched_dev_pivot['multi-family'].sum(),\
                                            sched_dev_pivot['total_units'].sum()]],
                   columns=['jurisdiction','single-family','multi-family','total_units'])
sched_dev_pivot = sched_dev_pivot.append(sched_dev_pivot_for_region, ignore_index=True)

In [ ]:
# format table for pretty printing
sched_dev_table = sched_dev_pivot.copy()
sched_dev_table['single-family'] = sched_dev_table['single-family'].map('{:,.0f}'.format)
sched_dev_table['multi-family'] = sched_dev_table['multi-family'].map('{:,.0f}'.format)
sched_dev_table['total_units'] = sched_dev_table['total_units'].map('{:,.0f}'.format) 

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as FF

init_notebook_mode(connected=True)

In [ ]:
# create table of scheduled development events
figure = FF.create_table(sched_dev_table)
figure.layout.update({'title': 'Scheduled Development Events'})
figure.layout.margin.update({'t':50, 'b':100})
iplot(figure, filename='subplot_table_vertical')

In [ ]:
import cufflinks as cf
# make stacked bar graph of scheduled development events
sched_dev_bar = sched_dev_pivot.copy()
sched_dev_bar = sched_dev_bar[sched_dev_bar.jurisdiction != 'Region'] # remove Region from bar chart
sched_dev_bar = sched_dev_bar.set_index(['jurisdiction'])
del sched_dev_bar['total_units']  
iplot(sched_dev_bar.iplot(asFigure=True,kind='bar', barmode='stack',title='Scheduled Development Events by Jurisdiction'))